In [38]:
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [39]:
electronics = pd.read_csv("All Electronics.csv")

In [40]:
pd.set_option('display.max_colwidth', None)

In [41]:
electronics.sample(2)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
8324,"Smart Shelter Adjustable Size Universal 100% Metal Speaker Stand Wall Mount, Speaker Holder/Bracket, Accommodate All Speak...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/41yLKkFJONL._AC_UL320_.jpg,https://www.amazon.in/Smart-Shelter-Adjustable-Universal-Speaker/dp/B07DKCMKF5/ref=sr_1_8327?qid=1679134163&s=electronics&sr=1-8327,3.6,641,₹699,"₹1,999"
518,"Redgear A-15 Wired Gaming Mouse with Upto 6400 DPI, RGB & Driver Customization for PC(Black)","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T1/images/I/61MUoISbzjL._AC_UL320_.jpg,https://www.amazon.in/Redgear-Gaming-Semi-Honeycomb-Windows-Gamers/dp/B08CHZ3ZQ7/ref=sr_1_524?qid=1679133681&s=electronics&sr=1-524,4.3,"14,823",₹499,₹799


In [42]:
electronics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            9600 non-null   object
 1   main_category   9600 non-null   object
 2   sub_category    9600 non-null   object
 3   image           9600 non-null   object
 4   link            9600 non-null   object
 5   ratings         9505 non-null   object
 6   no_of_ratings   9505 non-null   object
 7   discount_price  9116 non-null   object
 8   actual_price    9530 non-null   object
dtypes: object(9)
memory usage: 675.1+ KB


In [43]:
electronics.duplicated().sum()

0

In [44]:
electronics.isnull().sum()

name                0
main_category       0
sub_category        0
image               0
link                0
ratings            95
no_of_ratings      95
discount_price    484
actual_price       70
dtype: int64

In [45]:
electronics['actual_price'] = electronics['actual_price'].fillna("₹19,000")
electronics.isnull().sum()

name                0
main_category       0
sub_category        0
image               0
link                0
ratings            95
no_of_ratings      95
discount_price    484
actual_price        0
dtype: int64

In [46]:
electronics["#keywords"] = electronics['name'].str.lower()

In [47]:
electronics.head(1)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB Storage)","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJL._AC_UL320_.jpg,https://www.amazon.in/Redmi-Power-Black-128GB-Storage/dp/B09Y64H8VS/ref=sr_1_4?qid=1679133649&s=electronics&sr=1-4,4.0,965,"₹10,999","₹18,999","redmi 10 power (power black, 8gb ram, 128gb storage)"


In [48]:
electronics.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price', '#keywords'],
      dtype='object')

In [49]:
print(electronics["main_category"].value_counts())
print(electronics["sub_category"].value_counts())

tv, audio & cameras    9600
Name: main_category, dtype: int64
All Electronics    9600
Name: sub_category, dtype: int64


In [50]:
electronics.drop(['main_category','sub_category'],axis=1,inplace=True)

In [51]:
electronics

,name,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB Storage)",https://m.media-amazon.com/images/I/81eM15lVcJL._AC_UL320_.jpg,https://www.amazon.in/Redmi-Power-Black-128GB-Storage/dp/B09Y64H8VS/ref=sr_1_4?qid=1679133649&s=electronics&sr=1-4,4.0,965,"₹10,999","₹18,999","redmi 10 power (power black, 8gb ram, 128gb storage)"
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM, 128GB Storage)",https://m.media-amazon.com/images/I/71AvQd3VzqL._AC_UL320_.jpg,https://www.amazon.in/OnePlus-Nord-Lite-128GB-Storage/dp/B09WQYFLRX/ref=sr_1_5?qid=1679133649&s=electronics&sr=1-5,4.3,"113,956","₹18,999","₹19,999","oneplus nord ce 2 lite 5g (blue tide, 6gb ram, 128gb storage)"
2,"OnePlus Bullets Z2 Bluetooth Wireless in Ear Earphones with Mic, Bombastic Bass - 12.4 Mm Drivers, 10 Mins Charge - 20 Hrs...",https://m.media-amazon.com/images/I/51UhwaQXCpL._AC_UL320_.jpg,https://www.amazon.in/Oneplus-Bluetooth-Wireless-Earphones-Bombastic/dp/B09TVVGXWS/ref=sr_1_6?qid=1679133649&s=electronics&sr=1-6,4.2,"90,304","₹1,999","₹2,299","oneplus bullets z2 bluetooth wireless in ear earphones with mic, bombastic bass - 12.4 mm drivers, 10 mins charge - 20 hrs..."
3,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 128GB Storage) | 6000mAh Battery | Upto 12GB RAM with RAM Plus | Travel Adapte...",https://m.media-amazon.com/images/I/81I3w4J6yjL._AC_UL320_.jpg,https://www.amazon.in/Samsung-Mystique-Storage-Purchased-Separately/dp/B09TWGDY4W/ref=sr_1_7?qid=1679133649&s=electronics&sr=1-7,4.1,"24,863","₹15,999","₹24,999","samsung galaxy m33 5g (mystique green, 6gb, 128gb storage) | 6000mah battery | upto 12gb ram with ram plus | travel adapte..."
4,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM, 128GB Storage)",https://m.media-amazon.com/images/I/71V--WZVUIL._AC_UL320_.jpg,https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/dp/B09WQY65HN/ref=sr_1_8?qid=1679133649&s=electronics&sr=1-8,4.3,"113,956","₹18,999","₹19,999","oneplus nord ce 2 lite 5g (black dusk, 6gb ram, 128gb storage)"
...,...,...,...,...,...,...,...,...
9595,"PALAY® BTS Backpack For Boys Kpop BTS Bangtan School Backback For Student with Cable Vent, Backpack Travel Bag Backpack La...",https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/51WpV43F1ML._AC_UL320_.jpg,https://www.amazon.in/PALAY%C2%AE-Backpack-Bangtan-Backback-Student/dp/B09XMFBMCR/ref=sr_1_9598?qid=1679134236&s=electronics&sr=1-9598,4.1,37,"₹1,710","₹2,491","palay® bts backpack for boys kpop bts bangtan school backback for student with cable vent, backpack travel bag backpack la..."
9596,"MOCA iPad Air 5th Generation Case 2022 / iPad Air 4th 2020 Case 10.9 Inch, Slim Stand Hard Back Shell Protective Smart Cov...",https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/31lmoZ3XWQL._AC_UL320_.jpg,https://www.amazon.in/MOCA-Protective-Generation-Support-Charging/dp/B08SHY6FQV/ref=sr_1_9599?qid=1679134236&s=electronics&sr=1-9599,4.2,"1,133",₹475,"₹1,999","moca ipad air 5th generation case 2022 / ipad air 4th 2020 case 10.9 inch, slim stand hard back shell protective smart cov..."
9597,"ZEBRONICS Zeb-Sound Bomb 5 TWS V5.0 Bluetooth Truly Wireless in Ear Earbuds with Up to 22H Backup, Flash Connect, Splash P...",https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/61O7fCXFipL._AC_UL320_.jpg,https://www.amazon.in/Zebronics-Zeb-Sound-Bluetooth-Wireless-Earbuds/dp/B09NNNLBVD/ref=sr_1_9600?qid=1679134236&s=electronics&sr=1-9600,3.5,"3,177",₹962,"₹3,999","zebronics zeb-sound bomb 5 tws v5.0 bluetooth truly wireless in ear earbuds with up to 22h backup, flash connect, splash p..."
9598,LINQS® Original NXP Chip | Waterproof NFC Tag Sticker (Set of 5) | Smartrac Circus NTAG213 chip | for All NFC Phones | Vin...,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/71Sg1Qsf+4L._AC_UL320_.jpg,https://www.amazon.in/LINQS-Sticker-Smartrac-Circus-NTAG213/dp/B011CD6OVM/ref=sr_1_9601?qid=1679134236&s=electronics&sr=1-9601,3.6,210

In [52]:
electronics['#keywords'] = electronics['#keywords'].str.replace('''[^\w\d\s]''','', regex = True)

In [53]:
electronics["#keywords"]

0                                                                               redmi 10 power power black 8gb ram 128gb storage
1                                                                      oneplus nord ce 2 lite 5g blue tide 6gb ram 128gb storage
2          oneplus bullets z2 bluetooth wireless in ear earphones with mic bombastic bass  124 mm drivers 10 mins charge  20 hrs
3            samsung galaxy m33 5g mystique green 6gb 128gb storage  6000mah battery  upto 12gb ram with ram plus  travel adapte
4                                                                     oneplus nord ce 2 lite 5g black dusk 6gb ram 128gb storage
                                                                  ...                                                           
9595    palay bts backpack for boys kpop bts bangtan school backback for student with cable vent backpack travel bag backpack la
9596     moca ipad air 5th generation case 2022  ipad air 4th 2020 case 109 inch slim stand hard 

In [54]:
stemmer = PorterStemmer()

In [55]:
def stemming(text):
    words =[]
    for word in text.split(' '):
        words.append(stemmer.stem(word))
    return ' '.join(words)

In [56]:
text ="dancing in the dance floor while dancing he fell on the floow and dancer"
stemming(text)

'danc in the danc floor while danc he fell on the floow and dancer'

In [57]:
electronics["#keywords"] = electronics["#keywords"].apply(stemming)

In [58]:
electronics["#keywords"]

0                                                                            redmi 10 power power black 8gb ram 128gb storag
1                                                                    oneplu nord ce 2 lite 5g blue tide 6gb ram 128gb storag
2                oneplu bullet z2 bluetooth wireless in ear earphon with mic bombast bass  124 mm driver 10 min charg  20 hr
3            samsung galaxi m33 5g mystiqu green 6gb 128gb storag  6000mah batteri  upto 12gb ram with ram plu  travel adapt
4                                                                   oneplu nord ce 2 lite 5g black dusk 6gb ram 128gb storag
                                                                ...                                                         
9595    palay bt backpack for boy kpop bt bangtan school backback for student with cabl vent backpack travel bag backpack la
9596         moca ipad air 5th gener case 2022  ipad air 4th 2020 case 109 inch slim stand hard back shell protect smart cov


In [59]:
cv = CountVectorizer(max_features = 5000,stop_words='english',dtype=np.uint8)

In [60]:
cv.fit(electronics["#keywords"])

CountVectorizer(dtype=<class 'numpy.uint8'>, max_features=5000,
                stop_words='english')

In [61]:
vector = cv.transform(electronics["#keywords"]).toarray()

In [62]:
vector.shape

(9600, 5000)

In [63]:
similarity = cosine_similarity(vector)

In [64]:
del (vector) 
electronics.drop(["#keywords"], axis=1,inplace=True)

In [65]:
similarity.shape

(9600, 9600)

In [66]:
similarity[0]

array([1.        , 0.28603878, 0.07106691, ..., 0.        , 0.        ,
       0.08703883])

In [67]:
electronics['name'].sample(5)

3940    Wisely |Pack 2| For Fire Boltt Invincible Plus SmartWatch Screen Guard Scratch Guard Full Screen Protector Compatible for ...
6132                                                                    Samsung Galaxy A23 5G, Silver (8GB, 128GB Storage) with Offer
6118    Lenovo IdeaPad Slim 3 Intel Celeron N4020 4th Gen 15.6" (39.62cm) HD 220nits Anti-Glare (8GB/256GB SDD/Windows 11/Office 2...
2716    boAt Stone 620 Bluetooth Speaker with 12W RMS Stereo Sound, 10HRS Playtime, TWS Feature, IPX4, Multi-Compatibility Modes(G...
2577                                           Faber-Castell Erasable Crayon Tin Set - Pack of 12 (Assorted) + 2 FREE Gold and Silver
Name: name, dtype: object

In [68]:
product ="Nik case Silicon Cover for Realme Buds Air Pro/Air 2 (Soft|Silicon|Black)"
electronics[electronics['name']==product]

,name,image,link,ratings,no_of_ratings,discount_price,actual_price
8667,Nik case Silicon Cover for Realme Buds Air Pro/Air 2 (Soft|Silicon|Black),https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/4174hmb5xkL._AC_UL320_.jpg,https://www.amazon.in/Nik-case-Silicon-Cover-Realme/dp/B08P8Z2SK8/ref=sr_1_8670?qid=1679134185&s=electronics&sr=1-8670,3.8,"2,331",₹289,₹599


In [69]:
def recommender(product):
    product_index = electronics[electronics['name']==product].index[0]
    similarity_list = list(enumerate(similarity[product_index]))
    top_10_similar_product =sorted(similarity_list, key = lambda x:x[1],reverse=True)[1:11]
    for idx,similarly in top_10_similar_product:
        print(electronics.loc[idx]['name'])

In [70]:
recommender("Nik case Silicon Cover for Realme Buds Air Pro/Air 2 (Soft|Silicon|Black)")

Telca Case Cover Compatible with Realme Buds Air 3 NEO | ONLY Silicon Cover, NO Bluetooth (Buds Air 3 NEO, Black)
Telca Case Cover Compatible with Realme Air 3 ONLY (Silicon Cover)
Sounce Earphone Case Cover Compatible with Realme 3s Buds, Soft Silicone Skin Case Cover Shock-Absorbing Protective Case w...
Sounce Earphone Case Cover Compatible with Realme 3s Buds, Soft Silicone Skin Case Cover Shock-Absorbing Protective Case w...
Sounce Earphone Case Cover for Realme TechLife Buds T100, Soft Silicone Skin Case Cover Shock-Absorbing Protective Case wi...
KHR® Silicone Protective Case Cover Compatible for Oppo Enco Buds 2 Full Protection Soft Silicone Case Cover with Hook, Black
Amazon Brand - Solimo Back Cover Case for Realme 3 / Realme 3i | Compatible for Realme 3 / Realme 3i Back Cover Case | 360...
Sounce Case Silicone Cover, Compatible with Keychain for Oneplus Nord Buds Full Protection Shockproof Case Cover Anti Fall...
Sounce Case Silicone Cover, Compatible with Keychain for Oneplu

In [71]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [72]:
pickle.dump(electronics,open('data.pkl','wb'))